In [7]:
import nibabel as nib
import numpy as np
from sklearn.feature_selection import mutual_info_regression
import os

In [8]:
print(os.getcwd())

/Users/pabloprieto/Library/CloudStorage/OneDrive-Personal/Documentos/3rd_Year_Biomedical_Engineering_MENG/Summer_Studentship/Correlation_Templates


In [9]:
path_transformed_features_folder = "/Users/pabloprieto/Library/CloudStorage/OneDrive-Personal/Documentos/3rd_Year_Biomedical_Engineering_MENG/Summer_Studentship/Templates/Output_MSM/L21_L23_Curvature_Sulc_Warp.transformed_and_reprojected.func.gii"

In [10]:
path_original_features = "/Users/pabloprieto/Library/CloudStorage/OneDrive-Personal/Documentos/3rd_Year_Biomedical_Engineering_MENG/Summer_Studentship/Templates/Output_MSM/FeatureCombination"

In [38]:
data_transformed_features = nib.load(path_transformed_features_folder)
data_original_L21_features = nib.load(os.path.join(path_original_features,"fetal.week21.left.Curvature.Sulc.shape.gii"))
data_original_L23_features = nib.load(os.path.join(path_original_features,"fetal.week23.left.Curvature.Sulc.shape.gii"))

In [63]:
# GIFTI images can have multiple data arrays, so you need to specify which one you want to extract
curvature_transformed = data_transformed_features.darrays[0].data
curvature_original_L21_features = data_original_L21_features.darrays[0].data
curvature_original_L23_features = data_original_L23_features.darrays[0].data
# Checking if the sizes are the same (which they are)
print(curvature_transformed.shape)
print(curvature_original_L21_features.shape)
print(curvature_original_L23_features.shape)

has_nantr = np.isnan(curvature_transformed).any()
has_nan21 = np.isnan(curvature_original_L21_features).any()
has_nan23 = np.isnan(curvature_original_L23_features).any()

print(has_nantr,has_nan21,has_nan23)

(32492,)
(32492,)
(32492,)
False False False


In [64]:
# Specifying the sulcus feature
sulcus_original_L21_features = data_original_L21_features.darrays[1].data
sulcus_original_L23_features = data_original_L23_features.darrays[1].data
sulcus_transformed = data_transformed_features.darrays[1].data
# Checking if the sizes for the sulcus are the same
print(sulcus_transformed.shape)
print(sulcus_original_L21_features.shape)
print(sulcus_original_L23_features.shape)

has_nantr = np.isnan(sulcus_transformed).any()
has_nan21 = np.isnan(sulcus_original_L21_features).any()
has_nan23 = np.isnan(sulcus_original_L23_features).any()

print(has_nantr,has_nan21,has_nan23)

(32492,)
(32492,)
(32492,)
False False False


In [55]:

sulcus_original_L21_features = sulcus_original_L21_features.reshape(-1, 1)
sulcus_original_L23_features = sulcus_original_L23_features.reshape(-1, 1)
sulcus_transformed = sulcus_transformed.reshape(-1, 1)

mi_sulc_original_L21_to_original_L23 = mutual_info_regression(sulcus_original_L21_features, sulcus_original_L23_features.ravel())
mi_sulc_original_L21_to_transformed_L21L23 = mutual_info_regression(sulcus_original_L21_features, sulcus_transformed.ravel())
mi_sulc_original_L23_to_transformed_L21L23 = mutual_info_regression(sulcus_original_L23_features, sulcus_transformed.ravel())

print(mi_sulc_original_L21_to_original_L23)
print(mi_sulc_original_L21_to_transformed_L21L23)
print(mi_sulc_original_L23_to_transformed_L21L23)


[3.47763033]
[3.13394847]
[3.08510904]


In [56]:
curvature_original_L21_features = curvature_original_L21_features.reshape(-1, 1)
curvature_original_L23_features = curvature_original_L23_features.reshape(-1, 1)
curvature_transformed = curvature_transformed.reshape(-1, 1)

mi_curvature_original_L21_to_original_L23 = mutual_info_regression(curvature_original_L21_features, curvature_original_L23_features.ravel())
mi_curvature_original_L21_to_transformed_L21L23 = mutual_info_regression(curvature_original_L21_features, curvature_transformed.ravel())
mi_curvature_original_L23_to_transformed_L21L23 = mutual_info_regression(curvature_original_L23_features, curvature_transformed.ravel())

print(mi_curvature_original_L21_to_original_L23)
print(mi_curvature_original_L21_to_transformed_L21L23)
print(mi_curvature_original_L23_to_transformed_L21L23)

[2.37150999]
[2.06182581]
[1.91478724]


In [15]:
import numpy as np
from scipy.stats import spearmanr, kendalltau
from sklearn.feature_selection import mutual_info_regression

def print_correlations(x, y, name):
    # Pearson Correlation Coefficient
    pearson_corr = np.corrcoef(x, y)[0, 1]

    # Spearman's Rank Correlation Coefficient
    spearman_corr, _ = spearmanr(x, y)

    # Kendall's Tau
    kendall_corr, _ = kendalltau(x, y)

    # Mutual Information
    mi = mutual_info_regression(x.reshape(-1, 1), y.ravel())

    print(f"{name} Pearson Correlation: {pearson_corr}")
    print(f"{name} Spearman Correlation: {spearman_corr}")
    print(f"{name} Kendall Tau: {kendall_corr}")
    print(f"{name} Mutual Information: {mi[0]}")


In [ ]:
# Sulcus Correlations
print_correlations(sulcus_original_L21_features, sulcus_original_L23_features, "Sulcus L21 to L23")
print_correlations(sulcus_original_L23_features, sulcus_transformed, "Sulcus L23 to Transformed")

# Curvature Correlations
print_correlations(curvature_original_L21_features, curvature_original_L23_features, "Curvature L21 to L23")
print_correlations(curvature_original_L23_features, curvature_transformed, "Curvature L23 to Transformed")


In [72]:
if np.std(curvature_original_L21_features) == 0 or np.std(curvature_original_L23_features) == 0:
    print("One or both input arrays have no variation. Pearson correlation is not defined.")
else:
    # Calculate Pearson correlation
    pearson_corr = np.corrcoef(curvature_original_L21_features, curvature_original_L23_features)[0, 1]
    print(f"Pearson Correlation Original-Original: {pearson_corr}")

if np.std(curvature_original_L23_features) == 0 or np.std(curvature_transformed) == 0:
    print("One or both input arrays have no variation. Pearson correlation is not defined.")
else:
    # Calculate Pearson correlation
    pearson_corr = np.corrcoef(curvature_original_L23_features, curvature_transformed)[0, 1]
    print(f"Pearson Correlation Original-Transformed: {pearson_corr}")



if np.std(sulcus_original_L21_features) == 0 or np.std(sulcus_original_L23_features) == 0:
    print("One or both input arrays have no variation. Pearson correlation is not defined.")
else:
    # Calculate Pearson correlation
    pearson_corr = np.corrcoef(sulcus_original_L21_features, sulcus_original_L23_features)[0, 1]
    print(f"Pearson Correlation Original-Original: {pearson_corr}")

if np.std(sulcus_original_L23_features) == 0 or np.std(sulcus_transformed) == 0:
    print("One or both input arrays have no variation. Pearson correlation is not defined.")
else:
    # Calculate Pearson correlation
    pearson_corr = np.corrcoef(sulcus_original_L23_features, sulcus_transformed)[0, 1]
    print(f"Pearson Correlation Original-Transformed: {pearson_corr}")

Pearson Correlation Original-Original: 0.9866510860940401
Pearson Correlation Original-Transformed: 0.9645343162238491
Pearson Correlation Original-Original: 0.9991859242922988
Pearson Correlation Original-Transformed: 0.997297828749563


In [11]:
path_to_transformed_features = "/Users/pabloprieto/Library/CloudStorage/OneDrive-Personal/Documentos/3rd_Year_Biomedical_Engineering_MENG/Summer_Studentship/Templates/Output_MSM/L34_L36_Curvature_Sulc_Warp.transformed_and_reprojected.func.gii"

In [14]:
data_transformed_features = nib.load(path_transformed_features_folder)
data_original_L21_features = nib.load(os.path.join(path_original_features,"fetal.week34.curvature.sulc.shape.gii"))
data_original_L23_features = nib.load(os.path.join(path_original_features,"fetal.week34.curvature.sulc.shape.gii"))

In [22]:
import os
import nibabel as nib
import numpy as np
from scipy.stats import spearmanr, kendalltau
from sklearn.feature_selection import mutual_info_regression

def load_data_features(transformed_features,original_start_features,original_end_features,path_to_features="/Users/pabloprieto/Library/CloudStorage/OneDrive-Personal/Documentos/3rd_Year_Biomedical_Engineering_MENG/Summer_Studentship/Templates/Output_MSM/FeatureCombination",path_to_transformed_features="/Users/pabloprieto/Library/CloudStorage/OneDrive-Personal/Documentos/3rd_Year_Biomedical_Engineering_MENG/Summer_Studentship/Templates/Output_MSM"):
    full_path_transformed_features = os.path.join(path_to_transformed_features,transformed_features)
    full_path_original_start_features = os.path.join(path_to_features,original_start_features)
    full_path_original_end_features = os.path.join(path_to_features,original_end_features)

    data_transformed_features = nib.load(full_path_transformed_features)
    data_original_start_features = nib.load(full_path_original_start_features)
    data_original_end_features = nib.load(full_path_original_end_features)

    # GIFTI images can have multiple data arrays, so we to specify which one you want to extract.
    # By default we will extract curvature first and then sulcus
    curvature_transformed = data_transformed_features.darrays[0].data
    curvature_original_start_features = data_original_start_features.darrays[0].data
    curvature_original_end_features = data_original_end_features.darrays[0].data

    sulcus_transformed = data_transformed_features.darrays[1].data
    sulcus_original_start_features = data_original_start_features.darrays[1].data
    sulcus_original_end_features = data_original_end_features.darrays[1].data

    print_correlations(curvature_original_start_features, curvature_original_end_features, "Curvature L34 to L36")
    print_correlations(curvature_original_end_features, curvature_transformed, "Curvature L36 to Transformed")

    print_correlations(sulcus_original_start_features, sulcus_original_end_features, "Sulcus L34 to L36")
    print_correlations(sulcus_original_end_features, sulcus_transformed, "Sulcus L36 to Transformed")



In [23]:
load_data_features("L34_L36_Curvature_Sulc_Warp.transformed_and_reprojected.func.gii","fetal.week34.curvature.sulc.shape.gii","fetal.week36.curvature.sulc.shape.gii")

Curvature L34 to L36 Pearson Correlation: 0.9413626238391055
Curvature L34 to L36 Spearman Correlation: 0.9418675968389202
Curvature L34 to L36 Kendall Tau: 0.8066620759122813
Curvature L34 to L36 Mutual Information: 1.3265163993264553
Curvature L36 to Transformed Pearson Correlation: 0.9511005887317063
Curvature L36 to Transformed Spearman Correlation: 0.9493282341010186
Curvature L36 to Transformed Kendall Tau: 0.8172790527435182
Curvature L36 to Transformed Mutual Information: 1.3369075064022269
Sulcus L34 to L36 Pearson Correlation: 0.9893330625811212
Sulcus L34 to L36 Spearman Correlation: 0.9868669842706895
Sulcus L34 to L36 Kendall Tau: 0.9067539539757395
Sulcus L34 to L36 Mutual Information: 2.0759675903686494
Sulcus L36 to Transformed Pearson Correlation: 0.9903587709934233
Sulcus L36 to Transformed Spearman Correlation: 0.9917749962414921
Sulcus L36 to Transformed Kendall Tau: 0.9243438016750448
Sulcus L36 to Transformed Mutual Information: 2.163398064581168
